In [69]:
## for co2 emissions only

## Imports
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

## Define Functions

def get_g_values(a,tau):

    g_1=0
    g_0=0

    for i in range(len(a)):
        g1 = np.sum( a[i] * tau[i] * ( 1. - ( 1. + 100/tau[i] ) * np.exp(-100/tau[i]) ) )
        g0 = np.exp( -1 * np.sum( a[i] * tau[i] * ( 1. - np.exp(-100/tau[i]) ) ) / g1 ) 
        g_1=g_1+g1
        g_0=g_0+g0

    g1=g_1
    g0=g_0

    return g0, g1

    
def calculate_alpha(G,G_A,T,r,g0,g1,iirf100_max = False):

    iirf100_val=r[0]+r[1]*(G-G_A+r[2]*T+r[3]*G_A)
  
    if iirf100_max: # if a max is given in input, otherwise false

        if iirf100_val>iirf100_max:
            iirf100_val=iirf100_max
        
    alpha_new = g0 * np.exp(iirf100_val / g1)
        
    return alpha_new

def step_concentration(R_old,G_A,E,C0,alpha,a,tau,dt=1): #set time step to 1 year default

    decay_rate = dt/(alpha*tau)
    decay_factor = np.exp(-decay_rate)
    R_new = E * a / decay_rate * ( 1. - decay_factor ) + R_old * decay_factor # there shouldn't be a dt in the first decay rate
    G_A_new = np.sum(R_new) #summing across 4th dimension of matrix (like across columns or rows)
    C_new = C0 + (G_A_new + G_A_old) / 2 #concentration changes

    return C_new,R_new,G_A_new

def unstep_concentration(R_old,G_A,alpha,a,tau,dt=1):
    
    decay_rate = dt/(alpha*tau)
    decay_factor = np.exp( -decay_rate )
    E_new = (( G_A - np.sum(R_old*decay_factor,axis=-1) ) / np.sum( a / decay_rate * ( 1. - decay_factor ) ,axis=-1 )) #summed from back to front
    R_new = E_new[...,None] * a * 1/decay_rate * ( 1. - decay_factor ) + R_old * decay_factor #creating extra dimension to E in R_new

    return E,R_new

def return_empty_emissions(df_to_copy=False, start_year=1765, end_year=2500, timestep=1):
    
    if type(df_to_copy)==pd.core.frame.DataFrame:
        df = pd.DataFrame(index = df_to_copy.index,
                          columns=pd.MultiIndex.from_product([df_to_copy.columns.levels[0]])).fillna(0).apply(pd.to_numeric)
    else:

        df = pd.DataFrame(index=np.arange(start_year,end_year+1,timestep)).fillna(0).apply(pd.to_numeric)
        df.index.rename('Year',inplace=True)

    return df
    
def return_empty_forcing(df_to_copy=False, start_year=1765, end_year=2500, timestep=1):
    
    if type(df_to_copy)==pd.core.frame.DataFrame:
        df = pd.DataFrame(index = df_to_copy.index,
                          columns=pd.MultiIndex.from_product([df_to_copy.columns.levels[0]])).fillna(0).apply(pd.to_numeric)
        
    else:
        
        df = pd.DataFrame(index=np.arange(start_year,end_year+1,timestep))).fillna(0).apply(pd.to_numeric)
        df.index.rename('Year',inplace=True)
        
    return df


TypeError: can't multiply sequence by non-int of type 'numpy.float64'

In [70]:
def run_FaIR(emissions_in = False,
            concentrations_in = False,
            forcing_in = False,
            gas_parameters = get_gas_parameter_defaults(),
            thermal_parameters = get_thermal_parameter_defaults(),
            show_run_info = True,
            aer_concs_in = False):
    
## concentration driven scenario:
if input('concentrations diven: ')=='T'
    diagnosed_emissions = np.zeros((n_year))
    C[:] = #concentrations_in
    G_A = C
    G_A = (G_A-C0)/emis2conc
    RF = step_forcing(C=C,C0=PI_conc,f=f)
    diagnosed_emissions,R = unstep_concentration(R_old=0,G_A=G_A,alpha=alpha,a=a,tau=tau,C0=C0,emis2conc=emis2conc,dt=1)
    S,T = step_temperature(S_old=0,F=np.sum(RF),q=q,d=d,dt=1)
    for t in tqdm(np.arange(1,n_year),unit=' year'):
        G = np.sum(diagnosed_emissions)
        alpha[t] = calculate_alpha(G=G,G_A=G_A[t-1],T=np.sum(S)[np.newaxis],r=r,g0=g0,g1=g1)
        diagnosed_emissions[t],R = unstep_concentration(R_old=R,G_A=G_A[t],alpha=alpha[t,np.newaxis],a=a,tau=tau,C0=PI_conc,emis2conc=emis2conc,dt=1)
        S,T[t] = step_temperature(S_old=S,F=np.sum(RF[t])[np.newaxis]+ext_forcing[t],q=q,d=d,dt=1)

    C_out = concentrations_in.copy()
    E_out = pd.DataFrame(diagnosed_emissions,index = time_index)


## not concentration driven (emissions) scenario:
if input('concentrations driven: ')=='F'
    G = np.cumsum(emissions_in)
    C[0],R,G_A = step_concentration(R_old = 0,G_A_old = 0,alpha=alpha[0,np.newaxis],E=emissions[0,np.newaxis],a=a,tau=tau,C0=PI_conc,emis2conc=emis2conc,dt=1)
    RF[0] = step_forcing(C=C[0],PI_conc=PI_conc[0],f=f)
    S,T[0] = step_temperature(S_old=0,F=np.sum(RF[0])[np.newaxis]+ext_forcing[0],q=q,d=d,dt=1)

    for t in tqdm(np.arange(1,n_year),unit=' year'):
        alpha[t] = calculate_alpha(G=G[t-1],G_A=G_A,T=np.sum(S)[np.newaxis],r=r,g0=g0,g1=g1)
        C[t],R,G_A = step_concentration(R_old = R,G_A_old=G_A,alpha=alpha[t,np.newaxis],E=emissions[t,np.newaxis],a=a,tau=tau,C0=PI_conc,emis2conc=emis2conc,dt=1)
        RF[t] = step_forcing(C=C[t],C0=PI_conc,f=f)
        S,T[t] = step_temperature(S_old=S,F=np.sum(RF[t])[np.newaxis]+ext_forcing[t],q=q,d=d,dt=1)

    C_out = pd.DataFrame(C,index = time_index)
    E_out = emissions_in.copy()

    ext_forcing = np.zeros(np.sum(RF)) + ext_forcing
    RF = np.concatenate((RF,ext_forcing))
    RF = np.concatenate((RF,np.sum(RF)))

    alpha_out = pd.DataFrame(alpha,index = time_index)
    RF_out = pd.DataFrame(RF,index = time_index,columns=pd.MultiIndex.from_product(['Carbon Dioxide']+['External','Total']))
    T_out = pd.DataFrame(T,index = time_index)

    out_dict = {'C':C_out, \
                'RF':RF_out, \
                'T':T_out, \
                'alpha':alpha_out, \
                'Emissions':E_out }

SyntaxError: invalid syntax (614639836.py, line 4)

In [ ]:
## Define Variables ##

emissions=

C = np.zeros((n_year))
RF = np.zeros((n_year))
T = np.zeros((n_year))
alpha = np.zeros((n_year))
empty_array = np.zeros(n_year+1)

#Parameters for co2:

a=[0.2173000,0.2240000,0.2824000,0.2763000]
tau=[10**9,394.4000,36.54000,4.304000]
r=[33.90000,0.01880000,2.670000,0.000000]
PI_conc=278.0000
q=
d=

## Run some calculations ##

emissions = pd.DataFrame({'carbon_dioxide':empty_array},index=np.arange(n_year+1))
gas_parameters = get_gas_parameter_defaults()
gas_parameters = gas_parameters.reindex(emissions.columns,axis=1,level=1)

thermal_parameters = get_thermal_parameter_defaults()

g1,g0 = get_g_values(a,tau)

alpha_new = calculate_alpha(G=0,G_A=0,T=0,r=r,g0=g0,g1=g1)

C_new,R_new,G_A_new = step_concentration(R_old=0,G_A=G_A,E=E,C0=PI_conc,alpha=alpha_new.T,a=a,tau=tau,dt=1)

E,R_new = unstep_concentration(R_old=R,G_A=G_A,alpha=alpha_new,a=a,tau=tau,dt=1)

f = gas_parameters.loc['f1':'f3'].values

d,q = [thermal_parameters.loc[x] for x in ['d','q']]

In [64]:
pulse_emissions = return_empty_emissions(df_to_copy=False, start_year=0, end_year=100, timestep=1)

# add pulses in year 10 (the units are GtC, MtCH4 and MtN2O-N2):

pulse_emissions.loc[10] += 10
    
# Now generate a compatible forcing dataframe:

pulse_forcing = return_empty_forcing(pulse_emissions)

# And run the model!

pulse_run = run_FaIR(emissions_in=pulse_emissions,forcing_in=pulse_forcing,gas_parameters=test_gas_parameters,thermal_parameters=test_thermal_parameters)

/opt/anaconda3/envs/fairv200_install/lib/python3.11/site-packages/fair/fair_runner.py:127: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0).apply(pd.to_numeric)
/opt/anaconda3/envs/fairv200_install/lib/python3.11/site-packages/fair/fair_runner.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0).apply(pd.to_numeric)


NameError: name 'test_gas_parameters' is not defined